##Install Libraries

In [ ]:

!pip install stats
!pip install upgrade networkx

!pip install yahoofinancials
!pip install tslearn
!pip install yfinance
import math
import networkx.algorithms.community as nx_comm
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tslearn.metrics import dtw_path
from tslearn.metrics import dtw
from sklearn.preprocessing import normalize
import scipy
import seaborn as sns
import plotly.graph_objs as go
import random
from dateutil.relativedelta import relativedelta
from scipy import stats
import datetime
import yfinance as yf
from yahoofinancials import YahooFinancials
import networkx as nx
from itertools import combinations


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.6/127.6 kB 707.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stats: filename=stats-0.1.2a0-py3-none-any.whl size=24298 sha256=eec0eaf377327a810dc75b16797800aa3876833928036ee5d30eaf46ebba851f
  Stored in directory: /root/.cache/pip/wheels/c9/85/bc/3371b9bce1e4f7b8b638e8c968a4dbd74db171ee180c48f808
Successfully built stats
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 k

##Set-up functions


In [ ]:
def distance_dist(M):
    shape = M.shape
    M = M.reshape(-1,1)
    nonM = M[M != 0]
    series = nonM.tolist()
    data_sorted = np.sort(nonM)
    N = len(M)
    for i in range(len(data_sorted)):
        if i/N > 0.975: 
            nonM[series.index(data_sorted[i])] = 1
        elif i/N > 0.95:
            nonM[series.index(data_sorted[i])] = 0.1
        elif i/N > 0.9:
            nonM[series.index(data_sorted[i])] = 0.01
        else:
            nonM[series.index(data_sorted[i])] = 0.001
    M = np.zeros(shape)
    M[np.triu_indices(shape[0], 1)] = nonM
    return M

In [ ]:
def get_data(term = 'short'):
    tickers = pd.read_html(
        'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

    # Get the data for this tickers from yahoo finance
    if term == 'long':
        data = yf.download(tickers.Symbol.to_list(), interval = "1d",  start = '2019-11-1', end = '2023-4-17')['Close']
    elif term == 'mid':
      data = yf.download(tickers.Symbol.to_list(), interval = "1h",  start = '2022-5-20', end = '2023-4-25')['Close']
    else:
      data = yf.download(tickers.Symbol.to_list(), interval = "5m",  start = '2023-3-1', end = '2023-4-30')['Close']
      #data2 = yf.download(tickers.Symbol.to_list(), interval = "5m",  start = '2023-4-11', end = '2023-4-18')['Close']
      #data3 = yf.download(tickers.Symbol.to_list(), interval = "1m",  start = '2023-4-3', end = '2023-4-9')['Close']
     # data = pd.concat([data1,data2])
    null_sum = data.isnull().sum(axis = 0)
    for col in data.columns:
        if null_sum[col]>(len(data[col])/20):
            data = data.drop(labels = col, axis = 1)
            print (col)
        data = data.fillna(method='ffill')

    df_std = data.copy(deep = True)
    # apply the z-score method
    for column in df_std.columns:
        df_std[column] = (df_std[column] - df_std[column].mean()) / df_std[column].std()
        
    return (df_std, data)
#print(data.head())

In [ ]:
def get_data_capitalization():
    tickers = pd.read_html(
        'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
    FMP_key = '36f8d80afffeb9577beb82ec167451eb'
    symbols = []
    market_caps = []
    for item in tickers:
       
        response = requests.get(url(symbol = item, api_key = FMP_key)).json()
        symbols.append(response[0]['symbol'])
        market_caps.append(response[0]['marketCap'])
    cap = {}
    for i in range(len(symbols)):
        cap[symbols[i]]=market_caps[i]

    
    return cap

def url(symbol: str, api_key):
    return "https://financialmodelingprep.com/api/v3/market-capitalization/" + symbol + "?apikey=" + api_key


In [ ]:
def setMatrix(data):
  mat = []
  companies = data.columns
  check_comp = data.columns.to_list()
  for s1 in companies:
      check_comp.remove(s1)
      sim_vect = np.zeros(len(companies)-len(check_comp))
      for s2 in check_comp:
        sim_vect = np.append(sim_vect,(dtw(data[[s1]], data[[s2]])))
      mat.append(sim_vect)
  mat = np.array(mat)
  shape = mat.shape
  #mat.reshape(-1,1)
  #mat = normalize(mat)
  #mat.reshape(shape)
  return mat


In [ ]:
def dist(M):
    # Get the upper triangle of the distance matrix
    triu_indices = np.triu_indices_from(M, k=1)
    distances = M[triu_indices].tolist()
    # Calculate the percentiles of the distances
    #percentiles = np.percentile(distances, np.arange(0, 100.5, 0.5))

    # Convert the distances to percentiles
    p_5 = np.percentile(distances, 5)
    p_25 = np.percentile(distances, 25)
    p_50 = np.percentile(distances, 50)
    p_10 = np.percentile(distances, 10)
    percentile_values = np.zeros_like(distances, dtype=float)
    #distances_percentiles = [stats.percentileofscore(distances, d, 'mean') for d in distances]
    #distances_percentiles = [ np.nanpercentile(distances, d) for d in distances]
    i = 0
    for d in distances:
        if d <= p_5:
            percentile_values[i] = 1
        elif d <= p_10:
            percentile_values[i] = 0.7
        elif d <= p_25:
            percentile_values[i] = 0.5
        elif d <= p_50:
            percentile_values[i] = 0.1
        else:
            percentile_values[i] = 0.05
            #percentile_values[i] = 0.1*distances[i]
        i += 1
        #percentile_values[i] = val
        
    # Create a new matrix with the percentiles
    dist_matrix = np.zeros_like(M, dtype=float)
    dist_matrix[triu_indices] = percentile_values

    # Copy the values to the lower triangle to make the matrix symmetric
    #dist_matrix += dist_matrix.T

    return dist_matrix

In [ ]:
def dist_uncomplete(M):
    # Get the upper triangle of the distance matrix
    triu_indices = np.triu_indices_from(M, k=1)
    distances = M[triu_indices]

    # Calculate the percentiles of the distances
    #percentiles = np.percentile(distances, np.arange(0, 100.5, 0.5))

    # Convert the distances to percentiles
    percentile_values = np.zeros_like(distances, dtype=float)
    p_5 = np.percentile(distances, 5)
    p_25 = np.percentile(distances, 25)
    p_50 = np.percentile(distances, 50)
    p_10 = np.percentile(distances, 10)    
    i = 0
    for d in distances:
        if d <= p_5:
            percentile_values[i] = 1
        elif d <= p_10:
            percentile_values[i] = 0.5
        elif d <= p_25:
            percentile_values[i] = 0.1
        elif d <= p_50:
            percentile_values[i] = 0
        else:
            percentile_values[i] = 0
            #percentile_values[i] = 0.1*distances[i]
        i += 1
        #percentile_values[i] = val
        
    # Create a new matrix with the percentiles
    dist_matrix = np.zeros_like(M, dtype=float)
    dist_matrix[triu_indices] = percentile_values

    # Copy the values to the lower triangle to make the matrix symmetric
    #dist_matrix += dist_matrix.T

    return dist_matrix

In [ ]:
def set_graph(M, l, data):
    G = nx.from_numpy_array(M, create_using=nx.Graph)
    d = {i:l[i] for i in G.nodes()}
    G = nx.relabel_nodes(G, d)
    data = data.tail(2)
    last_price = {node:data[node][-1] for node in G.nodes()}
    variation = {node:(data[node][-1]-data[node][0])/(data[node][0]) for node in G.nodes()}
    nx.set_node_attributes(G, last_price, name='value')
    nx.set_node_attributes(G, variation, name='variation')
    return G

In [ ]:
def comm_var(G, comm, measure):
    #var = {i:0 for i in range(len(comm))}
    var =[]
    i = 0
    for c in comm:
        count = 0
        for company in c:
            count += measure[company]
        var.append(count/len(c))
        i+=1
    return var
        

In [ ]:
def weighted_moving_average(data, length):
    data = data.iloc[:length]
    vars = {}
    val = 0
    for company in data.columns:
        av = 0
        for i in range(len(data[company])-1):
            av += (2*i/(length*(length+1)))*(data[company][i+1]-data[company][i])/data[company][i]
            t= val
        vars[company]= av
    return vars

In [ ]:
def moving_av(data, length):
    data = data.iloc[:length]
    av = 0
    vars = {}
    for company in data.columns:
        for i in range(len(data[company])-1):
            av = av + (data[company][i+1]-data[company][i])/data[company][i]
        av = av/length
        vars[company]= av
    return vars

In [ ]:
def log_return(data, m=5):
    l = {}
    for company in data.columns:
        l[company] = np.log(data[company][5]/data[company][0])
    return l

In [ ]:
def update_node(G, node, comm, measure):
    i = 0
    for c in comm:
        if node in c:
          index = i
          break
        i+=1
    val = 0
    w = 0
    for company in comm[index]:
        if company != node:
            val = val+measure[company]*G.get_edge_data(node, company)['weight']
            w = w + G.get_edge_data(node, company)['weight']
    val = val/w
    return val 

In [ ]:
def update_node_mov(G, node, comm, measure):
    i = 0
    for c in comm:
        if node in c:
          index = i
          break
        i+=1
    val = 0
    w = []
    values = []
    for company in comm[index]:
        w.append(G.get_edge_data(node, company, default = {'weight':0})['weight'])
        values.append(measure[company])
    return weighted_average(values, w)

In [ ]:
def weighted_average(distribution, weights):
    return sum([distribution[i]*weights[i] for i in range(len(distribution))])/sum(weights)

In [ ]:
def detect_n_comm_outlier(G,measure, communities,n):
    buy = {}
    short = {}
    comp_out = ''
    average_measure = comm_var(G, communities, measure)
    multiplier = [v/max(average_measure) for v in average_measure]
    avg = sum(measure.values())/len(measure)
    j=0
    vals = {}
    for comm in communities:
        #if variation_comm[i]<0 and variation_comm[i]<-1.0*val:
        if average_measure[j]<0:
            vals =  {company:(measure[company]-update_node(G, company, communities, measure)) for company in comm}
            c = sorted(vals.items(), key=lambda item: item[1])
            for i in range(int(len(comm)*n)):
                #if c[i][1] > 0:
                short[c[i][0]] = abs(multiplier[j])
        #elif variation_comm[i]>val and variation_comm[i]>0:
        else:
            vals =  {company:(update_node(G, company, communities, measure)-measure[company]) for company in comm}
            c = sorted(vals.items(), key=lambda item: item[1])
            for i in range(int(len(comm)*n)):
                #if c[i][1] > 0:
                buy[c[i][0]] = abs(multiplier[j])
        j = j+1
    return buy, short

In [ ]:
def detect_comm_outlier(G,val,measure, communities):
    buy = {}
    short = {}
    comp_out = ''
    average_measure = comm_var(G, communities, measure)
    i=0
    comm_vals = {}
    for comm in communities:
        max = 0
        #if variation_comm[i]<0 and variation_comm[i]<-1.0*val:
        if average_measure[i]<0:
            for company in comm:
                val = measure[company]-update_node(G, company, communities, measure)
                if val>abs(avg):
                    comp_out = company
                    max = val
            if max !=0:
                short[comp_out] = max
        #elif variation_comm[i]>val and variation_comm[i]>0:
        else:
            for company in comm:
                val = update_node(G, company, communities, measure)-measure[company]
                if val>abs(avg):
                    comp_out = company
                    max = val
            if max !=0:
                buy[comp_out] = max
        i +=1
    return buy, short



In [ ]:
def detect_comm_outlier_mod(G,measure, communities):
    average_measure = comm_var(G, communities, measure)
    i=0
    buy = {}
    short = {}
    avg = sum(measure.values())/len(measure)
    for comm in communities:
        if avg>average_measure[i]: 
             for company in comm:
                val = measure[company]-update_node(G, company, communities, measure)
                if val >0:
                    short[company]= val
        if average_measure[i]>avg: 
             for company in comm:
                val = update_node(G, company, communities, measure)-measure[company]
                if val >0:
                    buy[company]= val
        i=i+1
    return buy, short

In [ ]:
def community_analysis_cont(G,variation, communities):
    strat = {}
    comp_out = ''
    i=0
    short = {}
    buy = {}
    for comm in communities:
        max = 0
        for company in comm:
          val = variation[company]-update_node(G, company, communities, variation)
          if val>=0:
              short[company] = val
          else:
              buy[company] = abs(val)
    return short, buy

In [ ]:
def gr_un(data, length):
    companies = data.columns.tolist()
    data = data.iloc[:length]
    matrix = setMatrix(data)
    dist_matrix = dist_uncomplete(matrix)
    G = set_graph(dist_matrix, companies, data)
    return G

In [ ]:
def gr(data, length):
    companies = data.columns.tolist()
    data = data.iloc[:length]
    matrix = setMatrix(data)
    dist_matrix = dist(matrix)
    G = set_graph(dist_matrix, companies, data)
    return G

##Community Analysis


In [ ]:
def compute_perm_adj(G, comm):
    G_perm = rearrange_graph(G, comm)
    A_perm = nx.adjacency_matrix(G_perm).todense()
    
    return A_perm

def rearrange_graph(G, comm):
    G_perm = G.copy(); G_perm.clear() 
    
    comm_copy = comm.copy(); 
    comm_copy.sort(key=len, reverse = True) 
    
    for comm_set in comm_copy:
        G_perm.add_nodes_from(list(comm_set))
    
    for edge in G.edges.data():
        G_perm.add_edges_from([edge])
        
    return G_perm

In [ ]:
def compute_asso_one_run(G, perm_comm, seed):

    G_c = nx_comm.louvain_communities(G, seed = seed) 
    #G_c = nx_comm.asyn_lpa_communities(G, weight='weight')
    G_comm = []
    for comm in G_c:
        G_comm.append(set(comm))
    # get a graph with permuted nodes but no edges
    G_dummy = rearrange_graph(G, perm_comm); G_dummy.remove_edges_from(list(G_dummy.edges))
    
    for comm in G_comm:
        comm = list(comm)
        G_dummy_edges = [(a, b) for idx, a in enumerate(comm) for b in comm[idx + 0:]]
        G_dummy.add_edges_from(G_dummy_edges)
    A_dummy = nx.adjacency_matrix(G_dummy).todense()
    
    return A_dummy

In [ ]:
def compute_asso(G, comm, n_runs, perm = True):
    A_asso = np.zeros((len(G.nodes), len(G.nodes)))
    
    if perm:
        for i in range(n_runs):
            A_asso += compute_asso_one_run(G, comm, i)
    else:
        for i in range(n_runs):
            A_asso += compute_asso_one_run(G, [set(G.nodes)], i)
    A_asso /= n_runs
    
    return A_asso

In [ ]:
def compute_perm_adj(G, comm):
    G_perm = rearrange_graph(G, comm)
    A_perm = nx.adjacency_matrix(G_perm).todense()
    
    return A_perm



##Non Temporal Strategies


In [ ]:
def strategy_moving_av(data, df, capital,m, iter, G):
    communities = nx_comm.louvain_communities(G)
    companies = data.columns.tolist()
    values = df.tail(len (data) - 300)
    revenue = {} 
    moving_av = moving_average(data.iloc[300-m:], m)
    variation = {node:(values[node][1]-values[node][0])/(values[node][0]) for node in companies}
    revenue = []
    for i in range (iter):
        buy, short = detect_comm_outlier_mod(G, 50 ,moving_av, communities)
        moving_av = moving_average(data.iloc[300-m+i:], m)
        variation = {node:(values[node][i+1]-values[node][i])/(values[node][i+1]) for node in companies}
        day_avg = sum(variation.values()) / len(variation)
        buy_vals = [1+value for value in buy.values()]
        var_buy = [variation[comp] for comp in buy.keys()]
        short_vals = [1+value for value in short.values()]
        var_short = [-1.0*variation[comp] for comp in short.keys()]
        var = var_buy+var_short
        vals = buy_vals+short_vals
        day_avg = sum(variation.values()) / len(variation)
        our_avg = 0
        if len (var)>0:
            our_avg = (weighted_average(var, vals))
            capital = (our_avg+1)*capital
        revenue.append([capital, day_avg, our_avg])
    return revenue


In [ ]:
def strategy_outliers_var(data, df, capital,iter, G, n):
    communities = nx_comm.louvain_communities(G)
    companies = data.columns.tolist()
    values =data.iloc[300:]
    revenue = {} 
    variation = {node:(data[node][300]-values[node][299])/(values[node][299]) for node in companies}
    total_revenue = 0
    revenue = []
    for i in range (iter):
        our_avg = 0
        buy, short = detect_n_comm_outlier(G,variation, communities, 3)
        variation = {node:(values[node][i+1]-values[node][i])/(values[node][i+1]) for node in companies}
        day_avg = sum(variation.values()) / len(variation)
        for company in buy.keys():
            our_avg += variation[company]
        for company in short.keys():
            our_avg = our_avg - variation[company]
        our_avg = our_avg/(len(short)+len(buy))
        day_avg = sum(variation.values())/len(variation)
        capital = capital*(1+our_avg) 
        revenue.append([capital, day_avg, our_avg])
    return revenue

In [ ]:
def strategy_outliers_w(data, df, capital,iter,m, G, n):
    communities = nx_comm.louvain_communities(G)
    companies = data.columns.tolist()
    values = data.iloc[300:]
    revenue = {} 
    measure = weighted_moving_average(data.iloc[300-m-1:], m)
    total_revenue = 0
    revenue = []
    for i in range (iter):
        our_avg = 0
        buy, short = detect_n_comm_outlier(G,measure, communities, 3)
        measure =  weighted_moving_average(data.iloc[300-m+i:], m)
        variation = {node:(values[node][i+1]-values[node][i])/(values[node][i]) for node in companies}
        day_avg = sum(variation.values()) / len(variation)
        for company in buy.keys():
            our_avg += variation[company]
        for company in short.keys():
            our_avg = our_avg - variation[company]
        our_avg = our_avg/(len(short)+len(buy))
        day_avg = sum(variation.values())/len(variation)
        capital = capital*(1+our_avg) 
        revenue.append([capital, day_avg, our_avg])
    return revenue

In [ ]:
def strategy_mult_var(data, df, capital,iter, G):
    communities = nx_comm.louvain_communities(G)
    companies = data.columns.tolist()
    values = data.iloc[300:]
    variation = {node:(data[node][300]-values[node][299])/(values[node][299]) for node in companies}    
    revenue = []
    for i in range (iter):
        our_avg = 0
        buy, short = detect_comm_outlier_mod(G, variation, communities)
        variation = {node:(values[node][i+1]-values[node][i])/(values[node][i]) for node in companies}
        day_avg = sum(variation.values()) / len(variation)
        for company in buy.keys():
            our_avg += variation[company]
        for company in short.keys():
            our_avg = our_avg - variation[company]
        our_avg= our_avg/(len(short)+len(buy))
        day_avg = sum(variation.values()) / len(variation)
        capital = capital*(1.0+our_avg)
        revenue.append([capital, day_avg, our_avg])
    return revenue


In [ ]:
def strategy_mult_mov(data, df, capital,iter,m, G):
    communities = nx_comm.louvain_communities(G)
    companies = data.columns.tolist()
    values = data.iloc[300:]
    measure = moving_av(data.iloc[300-m-1:], m)
    revenue = []
    for i in range (iter):
        our_avg = 0
        buy, short = detect_comm_outlier_mod(G, measure, communities)
        measure = moving_av(data.iloc[300-m+i:], m)
        variation = {node:(values[node][i+1]-values[node][i])/(values[node][i]) for node in companies}
        day_avg = sum(variation.values()) / len(variation)
        for company in buy.keys():
            our_avg += variation[company]
        for company in short.keys():
            our_avg = our_avg - variation[company]
        our_avg= our_avg/(len(short)+len(buy))
        day_avg = sum(variation.values()) / len(variation)
        capital = capital*(1.0+our_avg)
        revenue.append([capital, day_avg, our_avg])
    return revenue

In [ ]:
def strategy_mult_w(data, df, capital,iter,m, G):
    communities = nx_comm.louvain_communities(G)
    companies = data.columns.tolist()
    values = data.iloc[300:]
    measure = log_return(data.iloc[300-5:], 5)
    revenue = []
    for i in range (iter):
        our_avg = 0
        buy, short = detect_comm_outlier_mod(G, measure, communities)
        variation = {node:(values[node][i+1]-values[node][i])/(values[node][i]) for node in companies}
        measure = log_return(data.iloc[300-5+i:], m)
        day_avg = sum(variation.values()) / len(variation)
        for company in buy.keys():
            our_avg += variation[company]
        for company in short.keys():
            our_avg = our_avg - variation[company]
        day_avg = sum(variation.values())/len(variation)
        our_avg= our_avg/(len(short)+len(buy))
        capital = capital*(1.0+our_avg)
        revenue.append([capital, day_avg, our_avg])
    return revenue

In [1]:
def strategy_outliers_mov(data, capital,iter,m, G, n):
    communities = nx_comm.louvain_communities(G)
    companies = data.columns.tolist()
    values = data.iloc[300:]
    revenue = {} 
    measure = moving_av(data.iloc[300-m-1:], m)
    revenue = []
    for i in range (iter):
        s = 0
        b = 0
        our_avg = 0
        buy, short = detect_n_comm_outlier(G,measure, communities,n)
        measure = moving_av(data.iloc[300-m+i:], m)
        variation = {node:(values[node][i+1]-values[node][i])/(values[node][i]) for node in companies}
        day_avg = sum(variation.values())/len(variation)
        for company in buy.keys():
            our_avg += variation[company]
            b+=variation[company]
        for company in short.keys():
            our_avg = our_avg - variation[company]
            s = s-variation[company]
        if our_avg != 0:
            our_avg = our_avg/(len(short)+len(buy))
        capital = capital*(1+our_avg)
        if len(short)>0: 
            s=s/len(short)
        if len(buy)>0:
            b=b/len(buy)
        revenue.append([capital, day_avg, our_avg,b,s])
    return revenue

In [ ]:
def strategy_comp_log(data, df, capital,iter,m, G):
    communities = nx_comm.louvain_communities(G)
    companies = data.columns.tolist()
    values = data.iloc[500:]
    measure = log_return(data.iloc[499:])
    revenue = []
    for i in range (iter):
        our_avg = 0
        distribution = []
        weights = []
        buy, short = community_analysis_cont(G, measure, communities)
        variation = {node:(values[node][i+1]-values[node][i])/(values[node][i]) for node in companies}
        measure = log_return(data.iloc[500+i:])
        day_avg = sum(variation.values()) / len(variation)
        for company in companies:
            if company in buy.keys():
                distribution.append(variation[company])
                weights.append(buy[company])
            else:
                distribution.append(-1.0*variation[company])
                weights.append(short[company])
        w = sum(weights)
        for i in range(len(weights)):
            weights[i] = weights[i]/w
        our_avg = weighted_average(distribution, weights)
        day_avg = sum(variation.values()) / len(variation)
        capital = capital*(1+our_avg)
        revenue.append([capital, day_avg, our_avg])
    return revenue

In [ ]:
def strategy_comp_var(data, df, capital,iter, G):
    communities = nx_comm.louvain_communities(G)
    companies = data.columns.tolist()
    values = data.iloc[500:]
    variation = {node:(data[node][500]-data[node][499])/(data[node][499]) for node in companies}    
    revenue = []
    for i in range (iter):
        our_avg = 0
        distribution = []
        weights = []
        buy, short = community_analysis_cont(G, variation, communities)
        variation = {node:(values[node][i+1]-values[node][i])/(values[node][i]) for node in companies}
        day_avg = sum(variation.values()) / len(variation)
        for company in companies:
            if company in buy.keys():
                distribution.append(variation[company])
                weights.append(buy[company])
            else:
                distribution.append(-1.0*variation[company])
                weights.append(short[company])
        w = sum(weights)
        for i in range(len(weights)):
            weights[i] = weights[i]/w
        our_avg = weighted_average(distribution, weights)
        day_avg = sum(variation.values()) / len(variation)
        capital = capital*(1+our_avg)
        revenue.append([capital, day_avg, our_avg])
    return revenue

In [ ]:
def strategy_comp_mov(data, df, capital,iter,m, G):
    communities = nx_comm.louvain_communities(G)
    companies = data.columns.tolist()
    values = data.iloc[500:]
    measure = moving_av(data.iloc[500-m:], m)
    revenue = []
    for i in range (iter):
        our_avg = 0
        distribution = []
        weights = []
        buy, short = community_analysis_cont(G, measure, communities)
        variation = {node:(values[node][i+1]-values[node][i])/(values[node][i]) for node in companies}
        measure = moving_av(data.iloc[500-m+i:], m)
        day_avg = sum(variation.values()) / len(variation)
        for company in companies:
            if company in buy.keys():
                distribution.append(variation[company])
                weights.append(buy[company])
            else:
                distribution.append(-1.0*variation[company])
                weights.append(short[company])
        w = sum(weights)
        for i in range(len(weights)):
            weights[i] = weights[i]/w
        our_avg = weighted_average(distribution, weights)
        day_avg = sum(variation.values()) / len(variation)
        capital = capital*(1+our_avg)
        revenue.append([capital, day_avg, our_avg])
    return revenue

##Temporal strategies

In [2]:
def temp_strat(temp_G, data, capital, iter,m,n):
    communities_temp = []
    companies = data.columns.tolist()
    layers = len(temp_G)
    values = data.iloc[layers*100:]
    measure =  moving_av(data.iloc[layers*100-m-1:], m)
    revenue = []
    count = []
    for G in temp_G:
        comm = nx_comm.louvain_communities(G)
        communities_temp.append(comm)
    for i in range(iter):
        s= 0
        b = 0
        buy_temp={comp:True for comp in companies}
        short_temp={comp:True for comp in companies}
        variation = {node:(values[node][i+1]-values[node][i])/(values[node][i]) for node in companies}
        our_avg = 0
        j=0
        for G in temp_G:
            buy, short = detect_n_comm_outlier(G, measure, communities_temp[j],n)
            for company in companies:
                if company not in buy.keys():
                    buy_temp[company] = False
                if company not in short.keys():
                    short_temp[company] = False
            j=j+1
        measure =  moving_av(data.iloc[layers*100+i-m:], m)
        counter = 0
        for company in companies:
            if buy_temp[company] == True:
                our_avg = our_avg+variation[company]
                b = b+variation[company]
            if short_temp[company] == True:
                our_avg = our_avg-variation[company]
                s = s-variation[company]
        if sum(buy_temp.values())>0:
            b = b/sum(buy_temp.values())
        if sum(short_temp.values())>0:
            s = s/sum(short_temp.values())
        if sum(buy_temp.values())+sum(short_temp.values())>0:
            our_avg = our_avg/(sum(buy_temp.values())+sum(short_temp.values()))
        day_avg = sum(variation.values())/len(variation)
        capital = capital*(1+s)
        revenue.append([capital,our_avg, day_avg,s,b])
        count.append(counter)
    return revenue, count

In [ ]:
def temporal_graph(data, layers,n, length = 100):
    temp_G = []
    end = n*layers*length
    data = data.iloc[:end]
    for i in range(layers):
        data_layer=data.iloc[::-1*n*(i+1)]
        G = gr(data_layer, length)
        temp_G.append(G)
    return temp_G

In [ ]:
def final_strat(data,df,layers,iter, capital, l=200, n):
    re = iter/100
    revenue = []
    counter = []
    for i in range(int(re)):
        temp_G = temporal_graph(df.iloc[100*i:], layers,1,length = l)
        r, c = temp_strat(temp_G, data.iloc[100*i:], 100, 100, 10, n)
        revenue = revenue + r
        counter = counter + c
    return revenue, counter